<a href="https://colab.research.google.com/github/teerasitk/signalProcessingInFinance/blob/main/Assignment08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 8
Portfolio Optimization and Pair Trading Stategy

## Modules from Lecture 11

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from typing import Any, Union, Tuple

In [ ]:
def findOptPort(data_frames:pd.DataFrame, 
                target_ret:Any=None
                ) -> Tuple[float, float, np.ndarray]:
  """
    minimize half of port variance = 0.5 w^T\Sigma w
    such that
    1) Expected Return = target_ret (w^T\mu = target_ret)
    2) sum (w) = 1 (w^Tx1=1)
    :param data_frames: data_frames of simple returns
    :param target_ret: target return. None for minimum variance portfolio
    :return:
      r: expected annualized return
      vol: expected volatility
      w: weight on each stock in a portfolio
  """
  days_in_year = 365 # convert daily to yearly
  num_data, num_stocks = data_frames.shape 
  mu_vector  = data_frames.mean().values # mean vector
  cov_matrix = data_frames.cov().values # Sigma
  if target_ret is None: # minimum variance portfolio
    A = np.zeros((num_stocks + 1, num_stocks + 1)) 
    A[:num_stocks, :num_stocks] = cov_matrix 
    A[:num_stocks, -1] = -1
    A[-1, :num_stocks] = 1
    # A =[\Sigma | -1_N
    #     1_N^T      |0]
    # b = [0_N^T | 1]^T
    b = np.zeros(num_stocks + 1)
    b[:num_stocks] = 0
    b[-1] = 1
    x = np.linalg.solve(A, b)     
    w = x[:-1]
    r_min = x[-1] # daily return at minimum variance portfolio
    vol = np.sqrt(np.dot(np.dot(w.T, cov_matrix), w) * days_in_year)
    r = r_min * days_in_year # yearly return
    return r, vol, w  
  else:
    #  A = [Sigma| -1_N ,-\mu
    #       1_N^T|0,       0 
    #       \mu  |0,       0]
    #
    # b =[0_N,^T target_ret, 1]^T
    A = np.zeros((num_stocks + 2, num_stocks + 2))
    A[:num_stocks, :num_stocks] = cov_matrix
    A[:num_stocks, -1] = -1
    A[-1, :num_stocks] = 1
    A[:num_stocks, -2] = -mu_vector
    A[-2, :num_stocks] = mu_vector
    b = np.zeros(num_stocks + 2)
    b[:num_stocks] = 0
    b[-2] = target_ret/ days_in_year # convert to daily return
    b[-1] = 1
    x = np.linalg.solve(A, b)
    w = x[:num_stocks]
    vol = np.sqrt(np.dot(np.dot(w.T, cov_matrix), w) * days_in_year)
    r = target_ret
    return r, vol, w

In [ ]:
def buildEfficientFrontier(data_frames:pd.DataFrame,
                           plot_graph:bool = False
                           ) -> Tuple[np.ndarray, np.ndarray]:
  """
  compute the efficient frontier for a given 
  historical simple returns
    :param data_frames: data_frames of simple returns
    :param plot_graph: True to plot the efficient frontier
    :return: 
    mean and volatility of efficient fronteir
  """
  days_in_year = 365
  num_data, num_stocks = data_frames.shape  
  mu_vect  = data_frames.mean()
  cov_mat = data_frames.cov()
  stock_names = data_frames.columns  
  ret_stock = mu_vect * days_in_year
  vol_stock  = data_frames.std() * np.sqrt(days_in_year)
  r_max = ret_stock.max() * 3
  
  if plot_graph: # example stock to plot
    num_shows = int(0.1*num_stocks)
    showed_stocks = np.random.choice(stock_names, size=num_shows)
    vol_show = vol_stock[showed_stocks]
    mean_show = ret_stock[showed_stocks]
    plt.scatter(vol_show, mean_show, c='red')
    for stock in showed_stocks:
      plt.text(vol_show[stock]*1.02, mean_show[stock],stock)
  # find the efficient frontiers
  r_min, vol_min, w_min = findOptPort(data_frames) 
  mean_array = np.linspace(r_min, r_max, 1000)
  vol_array = np.zeros((1000,))
  for k, rk in enumerate(mean_array):
    r, vol, w = findOptPort(data_frames, target_ret =  rk)     
    vol_array[k] = vol
  if plot_graph:
    plt.plot(vol_array, mean_array)
    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.title(f"Return vs Volatility of top {num_stocks} SET Stocks")
    plt.grid()
    plt.show()
  return vol_array, mean_array

In [ ]:
import scipy.optimize as opt 
def findMarketPortfolio(data_frames: pd.DataFrame, 
                        rf:float
                        ) -> Tuple[np.ndarray, float, float]:
  """
    Find Market Portfolio for a given risk-free interest rate
    :param data_frames: data_frames of simple returns
    :param rf: risk-free interest rate
    :return:
      w_opt: Market portfolio 
      vol_opt: volatility of the market portfolio 
      ret_best: return of the market portfolio 
  """
  def sharpe_ratio(target_ret):
    ret, vol, w = findOptPort(data_frames, target_ret)
    return (ret - rf)/vol  
  r_max = -1e100
  days_in_year = 365
  num_data, num_stocks = data_frames.shape  
  r_min, vol_min, w_in = findOptPort(data_frames)
  best_solution = opt.minimize(lambda x: -sharpe_ratio(x), r_min)
  ret_best = best_solution['x'][0]
  _,vol_opt, w_opt = findOptPort(data_frames, target_ret= ret_best)
  return w_opt , vol_opt, ret_best


In [ ]:
def plotCapitalMarketLine(data_frames:pd.DataFrame,
                          w_market:np.ndarray, 
                          vol_market:float, 
                          ret_market:float,
                          rf:float):
  """
  Plot the capital market line
  :param data_frames: Simple return 
  :param w_market: Market Portfolio
  :param vol_mark: Market Portfolio volatility
  :param: ret_market: Maker Portfolio Return
  :parma: rf: risk-free interest
  """
  vols, rets = buildEfficientFrontier(ret, plot_graph=False)
  slope = (ret_market - rf)/vol_market
  ret_array_line = vols*slope + rf
  plt.plot(vols, rets, vols, ret_array_line)
  plt.scatter(vol_market, ret_market, c="red")
  plt.text(vol_market*1.01, ret_market, "Market Portfolio")
  plt.xlabel("volatility")
  plt.ylabel("return")
  plt.grid()
  title = "Capital Market Line with "
  for symbol in data_frames.columns:
    title += symbol +" "
  plt.title(title)
  plt.legend(["Efficient Frontier", "Capital Market Line"])
  plt.show()


In [ ]:
!pip install qpsolvers
from qpsolvers import solve_qp
def findOptPortLongOnly(data_frames: pd.DataFrame,
                        target_ret:float)->Tuple[float, float, np.ndarray]:
  """
    :param data_frames: data_frames of simple returns
    :param target_ret: target return. None for minimum variance portfolio
    :return:
      r: expected annualized return
      vol: expected volatility
      w: weight on each stock in a portfolio

  """
  days_in_year = 365
  num_data, num_stocks = data_frames.shape  
  mu_vector  = data_frames.mean().values
  cov_matrix = data_frames.cov().values
  num_stocks = mu_vector.shape[0]
  P = cov_matrix
  q = np.zeros((num_stocks,))
  G = np.ones((1, num_stocks))
  h = np.array([1])
  A = mu_vector.reshape(1,-1)
  b = np.array([target_ret/days_in_year])
  lb = np.zeros((num_stocks,))
  ub = np.ones((num_stocks,))
  w = solve_qp(P,q,G,h,A,b,lb,ub,verbose=True, solver='cvxopt')
  vol = np.sqrt(np.dot(np.dot(w.T, cov_matrix),w) *days_in_year)
  r = np.dot(mu_vector,w) * days_in_year
  return r, vol, w


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def buildEfficientFrontierLongOnly(data_frames:pd.DataFrame,
                                   plot_graph:bool = False
                                   )->Tuple[np.ndarray, np.ndarray]:
  """
    Build Efficient Frontier for long only portfolio
    :param data_frames: data_frames of simple returns
    :return:
    vol_array: volatility of efficinet frontier
    mean_array: return of efficinet frontier
  """
  days_in_year = 365
  num_data, num_stocks = data_frames.shape  
  mu_vect  = data_frames.mean().values  
  r_min = 0
  r_max = mu_vect.max()*days_in_year
  mean_array = np.linspace(r_min, r_max, 1000)
  vol_array = np.zeros((1000,))
  for k in range(1000):
    rk,volk, wk = findOptPortLongOnly(data_frames, target_ret=mean_array[k])
    vol_array[k] = volk
  if plot_graph:
    plt.plot(vol_array, mean_array)
    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.title(f"Return vs Volatility of {num_stocks} SET Stocks")
    plt.grid()
    plt.show()
  return vol_array, mean_array

## Modules from Lecture 12

In [ ]:
class STATUS:
  LONG = "long"
  SHORT = "short"
  NO_POSITION = "no position"

In [ ]:
from sklearn.linear_model import LinearRegression # build Linear Regression Line
from statsmodels.tsa.stattools import adfuller # ADF Test for statistionart 
def buildLinearRegression(y1, y2):
  lin_model = LinearRegression() # initialize linear regression
  y = y1.values
  x = y2.values.reshape(-1,1) # always need to be nx1
  lin_model.fit(x, y)
  error = y - lin_model.predict(x)
  statistics,p_value,*_ = adfuller(error) # get only statistic and p-value ignore the rest
  return lin_model, statistics, p_value


In [ ]:
# Note that z = p1 - a * p2 - b
# we cannot long or short b. Thus, we will long and short only p1 -a * p2
def longSpread(p1t, p2t, a, cash, trans_cost=0):
  if a > 0: # long p1 and short p2
    p1_actual_price = p1t * (1 + trans_cost) # pay more when long
    p2_actual_price = p2t * (1 - trans_cost) # get less when short
  else: # long both p1 and p2 
    p1_actual_price = p1t * (1 + trans_cost) # long P1
    p2_actual_price = p2t * (1 + trans_cost) # long P2
  spread_share = cash / (p1_actual_price + np.abs(a) * p2_actual_price) 
  # Do not use cash more than what we have 
  s1 = spread_share # always long
  cash -= s1 * p1_actual_price # cash pays for buying s1
  s2 = -a * spread_share # stock2 that we owe
  cash -= s2 * p2_actual_price # get from short s2
  return s1, s2, cash

def shortSpread(p1t, p2t, a, cash, trans_cost=0):
  if a > 0: # short p1 and long p2
    p1_actual_price = p1t * (1 - trans_cost) # pay more when long
    p2_actual_price = p2t * (1 + trans_cost) # get less when short
  else: # short both p1 and p2 
    p1_actual_price = p1t * (1 - trans_cost)
    p2_actual_price = p2t * (1 - trans_cost)
  spread_share = cash / (p1_actual_price  + np.abs(a) * p2_actual_price) 
  # Do not use cash more than what we have 
  s1 = -spread_share # alway shorts
  cash -= s1 * p1_actual_price # cash pays for buying s1
  s2 = a * spread_share # stock2 that we owe
  cash -= s2 * p2_actual_price # get from short s2
  return s1, s2, cash


def closePosition(p1t, p2t, s1, s2, a, status, trans_cost=0):
  sign_a = np.sign(a)
  if status == STATUS.LONG: # close from long spread
    cash1 = s1 * p1t * (1 - sign_a * trans_cost) # get less due to long
    cash2 = s2 * p2t * (1 + sign_a * trans_cost) # pay more due to short
  elif status == STATUS.SHORT: # close from short spread
    cash1 = s1 * p1t * (1 + sign_a * trans_cost) # get less due to short
    cash2 = s2 * p2t * (1 - sign_a * trans_cost) # pay more due to long
  else:
    raise ValueError(f"status can be either {STATUS.LONG} or {STATUS.SHORT}")
  return cash1 + cash2

In [ ]:
def pairTrade(p1, p2, z_th, a, b, init_wealth=1_000_000, trans_cost=0):
  status = STATUS.NO_POSITION
  stock1 = stock2 = 0
  cash = init_wealth
  wealth = pd.Series()
  signal = pd.Series()
  for k, (p1t, p2t, t_k) in enumerate(zip(p1,p2, p1.index)):
    e = p1t - a * p2t - b
    if status == STATUS.NO_POSITION:
      if e > z_th: # SHORT SELL Spread
        stock1, stock2, cash = shortSpread(p1t, p2t, a, cash, trans_cost)
        status = STATUS.SHORT
      elif e < -z_th: # Long 
        stock1, stock2, cash = longSpread(p1t, p2t, a, cash, trans_cost)
        status = STATUS.LONG
    elif status == STATUS.LONG:
      if e >= 0: # Close Long Position
        cash += closePosition(p1t, p2t, stock1, stock2, a, status, trans_cost)
        stock1 = stock2 = 0
        status = STATUS.NO_POSITION
    else: # STATUS == STATUS.SHORT
      if e <=0 : # Close short position
        cash += closePosition(p1t, p2t, stock1, stock2, a, status, trans_cost)
        stock1 = stock2 = 0
        status = STATUS.NO_POSITION
    if status == STATUS.NO_POSITION:
      signal[t_k] = 0
    elif status == STATUS.LONG:
      signal[t_k] = 1
    else:
      signal[t_k] = -1
    cur_wealth = p1t * stock1 + p2t*stock2 + cash
    wealth[t_k] = cur_wealth
    #print(f"{status}, {stock1}, {stock2}, {cash}")
  return wealth, signal

In [ ]:
reg_models = dict()
min_adf = np.inf
for col in prices.columns:
  x0 = prices[col]
  x1 = prices.drop(col, axis=1)
  lin_model = LinearRegression()
  lin_model.fit(x1,x0) 
  x0_hat = lin_model.predict(x1)
  error = x0 - x0_hat 
  adf, p_value, *_ = adfuller(error, maxlag=1)
  # print(col, adf, p_value)
  reg_models[col] = lin_model
  if min_adf > adf:
    best_model = lin_model
    min_adf = adf
    best_p_value = p_value
    target = col 
    opt_coef = lin_model.coef_
    opt_intercept = lin_model.intercept_
    best_x1 = x1
    best_x0 = x0
print(f"Best model is to predict {target} with ADF: {min_adf} and p_value: {best_p_value}")

# 1. Find Efficient Market Porfolio in 2021 from stocks in the list given below

In [ ]:
set50 = ['ADVANC', 'AOT', 'AWC', 'BANPU', 'BBL', 'BDMS', 'BEM', 'BGRIM', 'BH', 
         'BLA', 'BTS', 'CBG', 'CPALL', 'CPF', 'CPN', 'CRC', 'DTAC', 'EA', 
         'EGCO', 'GLOBAL', 'GPSC', 'GULF', 'HMPRO', 'INTUCH', 'IRPC', 'IVL', 
         'JMART', 'JMT', 'KBANK', 'KCE', 'KTB', 'KTC', 'LH', 'MINT', 'MTC', 
          'OSP', 'PTT', 'PTTEP', 'PTTGC', 'SAWAD', 'SCC', 'SCGP',
         'TIDLOR', 'TISCO', 'TOP', 'TRUE', 'TTB', 'TU']

In [ ]:
# Code is here

## Solution: Find the top 5 stocks in the market portfolio. Here, we only consider the magnitude 
1. _____ Position:_____
2. _____ Position:_____
3. _____ Position:_____
4. _____ Position:_____
5. _____ Position:_____

# 2. Do you think it is possible to construct the market portfolio? why?

## Solution: 

#3. Plot the Capital Market Line where $r_f=2\%$

In [ ]:
# Add your code here

# 4. Find the Beta of all stock against the market portfolio. 
Report Stock with higest Beta, and all
stocks with negative beta

In [ ]:
# code here

## Solution:
Highest Beta: ______

Negative Beta:

  Stock1: ____ Beta:____

  ... 

#5. Use the set50 indexin set50Index2022.xlsx as the market portfolio. Compute Beta of all stocks. Find the highest Beta and all Negative one.

In [ ]:
# Code is here

## Solution:
Highest Beta: ______

Negative Beta:

  Stock1: ____ Beta:____

  ... 

# 6. We will use Data in 2017-2021 to find 1 pair trade of set 50.
The criteria in choosing the pair are given below
1. For a spread defined as $ɛ_t = y_{1t} - \gamma\times y_{2t} - \mu$, we want stock with negative $\gamma$, i.e., $\gamma < 0$ so that we will take long and short positions on both stock when trading on the spread.
2. $p$-value must be the smallest among pairs in (1).

In [ ]:
# Code is here

## Solution: This pair is (1) ___ and (2) ___. Here, we will use (2) to predict (1)

# 9. Perform Pair Trade with both Long and Short Spread with $N/T = 0.10$ and transcation cost = 1%




In [ ]:
# code here

## Solution:
* Gain(or Loss) at the end of 2021:______________.
* Number of Long: ______ and Short: _____ during 5 years.

# 10. Perform Pair Trade with **Long** Only on the Spread with $N/T = 0.1$

In [ ]:
# build your own long only spread trading

In [ ]:
# Other codes

## Solution:
* Gain(or Loss) at the end of 2021:______________.
* Number of Long: ___________ during 5 years.

# 11. Use the same spread function to trade on 2022 stocks, and repeat 9 and 10 

In [ ]:
# code here

## Solution:
* Gain (Loss) in 2022 with Long and short: ____
* Gain (Loss) in 2022 with Long Only: ____

# 12. In your opinion, do you think whether we should trade on the stock pair with negativen $\gamma$ or not? Provide the reason

## Solution:_______